In [2]:
from collections import defaultdict
import pandas as pd

# Read CSV file
file_path = "Mov preparacion 02.2025.csv"
df = pd.read_csv(file_path)

# Clean column names (remove spaces)
df.columns = df.columns.str.strip()

# Filter rows:
df = df[~df["Ubic.proc."].astype(str).str.startswith("4")]
df = df[df["Ubic.proc."].astype(str).str.endswith("10")]

# Ensure 'CtdTeóricaDesde' is numeric
df["CtdTeóricaDesde"] = pd.to_numeric(df["CtdTeóricaDesde"], errors="coerce")

# Total unique products
total_products = df["Material"].nunique()

# Create base tuples
matrix_data = list(zip(df["Material"], df["CtdTeóricaDesde"], df["Texto breve de material"], df["Ubic.proc."]))

# Group by material
grouped_data = defaultdict(lambda: [0, 0, "", ""])
for material, quantity, description, location in matrix_data:
    if pd.notnull(quantity):
        grouped_data[material][0] += quantity
        grouped_data[material][1] += 1
        grouped_data[material][2] = description
        grouped_data[material][3] = location

# Construction and initial sorting by frequency and quantity
grouped_matrix = [
    (material, total, count / total_products, desc, loc)
    for material, (total, count, desc, loc) in grouped_data.items()
]
grouped_matrix.sort(key=lambda x: (x[2], x[1]), reverse=True)

# Extract original locations
extracted_locations = [loc for _, _, _, _, loc in grouped_matrix]

# Custom sorting functions
def get_sub_block(location):
    try:
        return int(location.split('-')[1])
    except:
        return 999

def filter_and_sort(location_list, prefix, max_value=None, condition='<='):
    result = []
    for loc in location_list:
        if loc.startswith(prefix):
            value = get_sub_block(loc)
            if max_value is None or (condition == '<=' and value <= max_value) or (condition == '>' and value > max_value):
                result.append(loc)
    return sorted(result, key=lambda x: get_sub_block(x))

# Apply sorting rules
sorted_locations = []
sorted_locations += filter_and_sort(extracted_locations, "600", 48, "<=")
sorted_locations += filter_and_sort(extracted_locations, "620")
sorted_locations += filter_and_sort(extracted_locations, "660", 43, "<=")
sorted_locations += filter_and_sort(extracted_locations, "680", 43, "<=")
sorted_locations += filter_and_sort(extracted_locations, "600", 48, ">")
sorted_locations += filter_and_sort(extracted_locations, "630")
sorted_locations += filter_and_sort(extracted_locations, "660", 43, ">")
sorted_locations += filter_and_sort(extracted_locations, "680", 43, ">")
sorted_locations += filter_and_sort(extracted_locations, "690")
sorted_locations = list(dict.fromkeys(sorted_locations))

# Assignment of ordered locations
final_products = []
for (material, total, frequency, desc, _), new_location in zip(grouped_matrix, sorted_locations):
    final_products.append([material, total, frequency, desc, new_location])

# -------------------------
# CORRECTED RELOCATION LOGIC (ONLY CHANGE LOCATIONS)
# -------------------------

def parse_location(location):
    parts = location.split("-")
    return parts[0], int(parts[1]), parts[2]

def contains_brand(description, brands):
    description_upper = description.upper()
    return any(brand in description_upper for brand in brands)

key_brands = ["GRANINI", "JUVER", "MINUTE MAID"]

# First pass: Identify all conflicts
conflicts = []
for i in range(len(final_products)):
    current_material, _, _, current_description, current_location = final_products[i]
    current_block, current_zone, _ = parse_location(current_location)

    if not contains_brand(current_description, key_brands):
        continue

    # Search for conflicts in close neighbors
    for offset in [-2, -1, 1, 2]:
        j = i + offset
        if 0 <= j < len(final_products):
            neighbor_material, _, _, neighbor_description, neighbor_location = final_products[j]
            neighbor_block, neighbor_zone, _ = parse_location(neighbor_location)

            if (current_block == neighbor_block and
                abs(current_zone - neighbor_zone) == 2 and
                contains_brand(neighbor_description, key_brands)):

                # Register conflict (smaller index first to avoid duplicates)
                conflicts.append(tuple(sorted((i, j))))

# Remove duplicates
conflicts = list(set(conflicts))

# Second pass: Resolve conflicts by exchanging only locations
for i, j in conflicts:
    # Find non-conflicting neighbor to exchange locations
    for swap_offset in [-1, 1]:
        swap_index = i + swap_offset
        if (0 <= swap_index < len(final_products)) and not contains_brand(final_products[swap_index][3], key_brands):
            # Exchange ONLY the locations
            final_products[i][4], final_products[swap_index][4] = final_products[swap_index][4], final_products[i][4]
            break
    else:
        # If no suitable neighbor found, try with the other end
        for swap_offset in [-1, 1]:
            swap_index = j + swap_offset
            if (0 <= swap_index < len(final_products)) and not contains_brand(final_products[swap_index][3], key_brands):
                # Exchange ONLY the locations
                final_products[j][4], final_products[swap_index][4] = final_products[swap_index][4], final_products[j][4]
                break

# -------------------------
# SHOW FINAL RESULTS
# -------------------------

print("\nRelocated matrix (only locations were changed to resolve conflicts):\n")
print(f"Total different products: {total_products}\n")
print("{:<10} {:<10} {:<20} {:<30} {:<15}".format("Material", "Quantity", "Frequency", "Description", "Location"))
print("-" * 85)
for material, total, freq, desc, location in final_products:
    print("{:<10} {:<10.1f} {:<20.4f} {:<30} {:<15}".format(material, total, freq, desc[:28], location))

# Optional: Save results to CSV
result_df = pd.DataFrame(final_products,
                        columns=["Material", "Quantity", "Frequency", "Description", "Location"])
result_df.to_csv("relocation_results_corrected.csv", index=False)


Relocated matrix (only locations were changed to resolve conflicts):

Total different products: 381

Material   Quantity   Frequency            Description                    Location       
-------------------------------------------------------------------------------------
0RF0161    9475.0     1.4961               COCACOLA LATA33 C24 (SIN HC8   600-004-10     
0LT0235    1692.0     1.4803               LA LEVANTINA AVENA ESP.HOST    600-006-10     
0RF0187    10777.0    1.4593               COCACOLA ZER LATA33 C24 (SIN   600-007-10     
0LT0090    1148.0     1.2756               LETONA SEMI SIN LACTOSA 1L P   600-008-10     
ED13LT     1594.0     1.2598               ESTRELLA DAMM 1/3 LATA         600-009-10     
EC13P6     929.0      1.1339               DAURA DAMM 1/3 SR PACK CESTA   600-011-10     
0LT0034    2542.0     1.0420               LETONA SEMI ESPEC HOSTELERIA   600-012-10     
0ZU0020    686.0      1.0184               GRANINI MELOCOTON  20CL 24U    600-013-10     
0Z